## Better data create

In [48]:
import tensorflow as tf
import numpy as np

In [29]:
with tf.variable_scope("ifyouwantyou11"):
    sample = "if you want you" # set을 통해서 중복값을 제거한 후에 리스트로 만듦
    list_char = list(set(sample))
    dic_char = {c:i for i, c in enumerate(list_char)}

    sample_idx = [dic_char[i] for i in sample]
    x_data = [sample_idx[:-1]] # 끝에 하나 덜들어가고
    y_data = [sample_idx[1:]] # 처음 하나 덜들어감
                             # batch 때문에 겉에 [] 더 씌움

    #parameters
    hidden_size = len(list_char)
    sequence_length = len(sample_idx) -1
    learning_rate=0.1
    dic_size = len(dic_char)
    batch_size = 1
    num_classes = len(dic_char)

    X = tf.placeholder(tf.int32, [None, sequence_length])
    Y = tf.placeholder(tf.int32, [None, sequence_length])

    X_one_hot = tf.one_hot(X, num_classes) # 알아서 차원이 하나 늘어나기 때문에
    # reshpae 해줄 필요가 없음.
    
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=
                                       initial_state, dtype=tf.float32)
    
    weights = tf.ones([batch_size, sequence_length])
    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs,
                                targets = Y, weights = weights)
    loss = tf.reduce_mean(sequence_loss)
    
    train = tf.train.AdamOptimizer(learning_rate=learning_rate). minimize(loss)
            # tan 함수 사용 => 각 cost 함수가 convex 함수이기 때문에
            
    prediction = tf.argmax(outputs, 2) # argmax 를 하면 차원이 줄어들게 됨


In [30]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(3000):
        p, l, _ = sess.run([prediction, loss, train], feed_dict={X:x_data,Y:y_data})
        
        if step%300==0:
            p_str = [list_char[i] for i in np.squeeze(p)]
            # 차원이 1인 rank 를 없애줌
            p_str="".join(p_str)
            print(f"step : {step:04}, prediction : {p_str}")

step : 0000, prediction : yynuannuuuuuuu
step : 0300, prediction : f you wnnt you
step : 0600, prediction : f you wnnt you
step : 0900, prediction : f you wnnt you
step : 1200, prediction : f you wnnt you
step : 1500, prediction : f you wnnt you
step : 1800, prediction : f you wnnt you
step : 2100, prediction : f you wnnt you
step : 2400, prediction : f you wnnt you
step : 2700, prediction : f you wnnt you


## long sentece

In [35]:
sentence = ("if you want to build a ship, don't drum up people together to "
           "collect wood and don't assign them tasks and work, but rather "
           "teach them to long for endless immensity of the sea.")
# 한칸 씩 움직이면서 x_data 로 설정해서 정확도를 높임

In [60]:
char_list = list(set(sentence))
char_dic = {c:idx for idx, c in enumerate(char_list)}
x_data = []
y_data = []

#data
seq_length = 10
for i in range(len(sentence)-seq_length):
    x_str = sentence[i:seq_length+i]
    y_str = sentence[i+1:seq_length+i+1]
    
    x_str = [char_dic[c] for c in x_str]
    y_str = [char_dic[c] for c in y_str]
    
    x_data.append(x_str)
    y_data.append(y_str)

#parameters
input_size = len(char_dic)
hidden_size = len(char_dic)
nb_classes = len(char_dic)
batch_size = len(x_data)

with tf.variable_scope("sentence10"):
    X = tf.placeholder(tf.int32, [None, seq_length])
    Y = tf.placeholder(tf.int32, [None, seq_length])
    x_one_hot = tf.one_hot(X, nb_classes)
    
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype= tf.float32)
    
    weights = tf.ones([batch_size, seq_length])
    seq_cost = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets = Y, weights = weights)
    cost = tf.reduce_mean(seq_cost) # mean 은 어떤 형태던지 모든 값의 mean 을 구함!!!
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
    
    prediction = tf.argmax(outputs,2)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(3001):
        c, _ = sess.run([cost, optimizer], feed_dict = {X:x_data, Y:y_data})
        p = sess.run(prediction, feed_dict = {X:x_data})
        if step%300==0:
            p_str = [char_list[int(i)] for seq in p for i in seq]
            p_str = "".join(p_str)
            print(f"step: {step}, \ncost : {c:.6f}, \nprediction : {p_str}")
    
    
    

step: 0, 
cost : 3.210700, 
prediction : tto othohhoo othohhha cthhhhhh cttthhhooctathhhoooaaohhhhhooaohhhohooonhhhooooohhhhhhoohhhd ooooohhh ahoohhhhhaooohhhhhh toohhhhhhcorttthhhharahhhhhhhraht ahhhhatt ahhhhhtt a hhhhht a hhhhhh ahhhhhhhhahhhhhhhhhha hhh hhoa hth hooh ttt t oh ttt o oh ot  o od ooo oooo ooo a ot oooha ot ooohh ttt oohhhctttoohhhhd  oohhhhhr aohhhhhh aohhhhhhha hhhahhho  ataahoootttaahooooataaoooooonaaoooooooaaooooooooataoooooooooooooooooaooooooooooocoetooooccottoooooctetoooooooetooooohhttaoooohhhcaoooohhhhaoooohhhhh ttthhhhhhctctthhhhhechhhhhhhochhhhhhhoh hhhhhhohhttthhhohhoctahhohhottahoohhtttaooohootth tohtttthhcoettttthhahttttthhohtttttttoocttttttooottttthoooottthhooooochhhooooohhhhooooohh aoooooohoaootoo hoonttto hooocttt h oooc a h oood ahh ooohdahh oooh ohh oooh ood oooo ooo a oh oohha ot oohhh ttt ohhhhctttohhhhhd  ohhhhhhr ahhhhhhh ahh hhhhhah  hhhhhhh  tnhhhhh  tththhhh tththhhhotthohhhhohetoohhoohhdoohhoohhhaohhhohhhh hhtohhhhhtttoohhhhhctaohhhhhhnaohh  hh

step: 1500, 
cost : 1.865535, 
prediction : g you aantotou aant tou aant t u aant tonlaant to  want to otont to ouont to ouint to ouilt to ouild to auild to luild a  luild a nluild a ttutld a thuild a thi ld a ssipg  a thip,e a ssip, od ssip, dtnssip, donthip, donthip  don'iip  don'tet  don't g  don't d, don't dr don't druton't drumon't drum ndt drum ttt drum tptaarum tp  toum tp ptoum tp peo m tp pepum tp pepp  up pepp  tm peop etp peop e   pepp e t,peop e totoop e to ,op e to enrle to etn e to eth,e to ethee to ethernto ether to  ther t   ther ton ther to ether te cnoer to co em te colem te colln te colleute collecto lollect  lollect nlollect wtollect woollect woonlect woodeect woodoe t wood ant wood anohtood and tood and tood and dord and dond and donn and don'od d d n'ttnd d n't nt don't at don't a odon't asston't dssion't dssigndt dssigntt dssign taassign t tssign thtnsign thenhign themiign them ign them tgn them taetthem tastthem tasstoem tosss em tossssem tossssin tossssin tosss

In [61]:
char_list = list(set(sentence))
char_dic = {c:idx for idx, c in enumerate(char_list)}
x_data = []
y_data = []

#data
seq_length = 10
for i in range(len(sentence)-seq_length):
    x_str = sentence[i:seq_length+i]
    y_str = sentence[i+1:seq_length+i+1]
    
    x_str = [char_dic[c] for c in x_str]
    y_str = [char_dic[c] for c in y_str]
    
    x_data.append(x_str)
    y_data.append(y_str)

#parameters
input_size = len(char_dic)
hidden_size = len(char_dic)
nb_classes = len(char_dic)
batch_size = len(x_data)

with tf.variable_scope("sentence11"):
    X = tf.placeholder(tf.int32, [None, seq_length])
    Y = tf.placeholder(tf.int32, [None, seq_length])
    x_one_hot = tf.one_hot(X, nb_classes)
    
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype= tf.float32)
    
    weights = tf.ones([batch_size, seq_length])
    output = tf.argmax(outputs, 2)
    seq_cost = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets = Y, weights = weights)
    cost = tf.reduce_mean(seq_cost) # mean 은 어떤 형태던지 모든 값의 mean 을 구함!!!
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
    
    prediction = tf.argmax(outputs,2)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(3001):
        c, _ = sess.run([cost, optimizer], feed_dict = {X:x_data, Y:y_data})
        p = sess.run(prediction, feed_dict = {X:x_data})
        if step%300==0:
            p_str = [char_list[int(i)] for seq in p for i in seq]
            p_str = "".join(p_str)
            print(f"step: {step}, \ncost : {c:.6f}, \nprediction : {p_str}")


step: 0, 
cost : 3.214066, 
prediction : oooon dondotdn dondotdn dondoo   doodooon doodooootttodooooottodoooooeoodoooooeondoooooeoodoooooeeoeoooooeeoeotoedeeoeooooottoeoonnnttoeodndtttaeooodnttaaooodneto ododnenoeodddneneeodndnnnnnodndnnnnnntndnenennnndnenendnntaeeeadnnnaiaeadnnndeoeddonndoodddnnndoddddnnndodondnnndodootnnddodoooonddodoooonddodoooooddoooooooododoooooooooooooooootnooodoodeooooooodoeoatttatdeettttdtdeeetttdtdeeeettdteeeeeetdtdeeeeeedteeeeeeeetdeeeeeeeedeeeeeeeee   eeeeeeenneeeeeoeedeeeeeoeeee eeeoeoee toeoeoeeetoeoeoeeeeoooeoeeeoonnnoeeeooeoeoeeeooeo oeeeooeoeoeeoooooeoeeotoeeeoe otoooooeeotoooooeeetoedeneeeeoooooeeeeonnoneeeeootoneeeeoooooeeeeoooonee eoooooee eooooooe eoooooon ooooooonnooooooonnnooooonnnnnttnnnnnnnnornnnnnnnnnnnnnnnnnnnnnndnnnnnodndnnnnnntndodnnnnnndodnnnnoddodonndodnodnnndodnntnnddodnnnonddoddnnnnddodoonnnddooooaooododooaaoddooonnnnddotnnnnnddoennnnnddoeeaaaaadoeeeaaoodoeeeeooodoeeeeoottoeeeeooddoeeeeoootoeeeeoooeoeeetoooeaeeetooonnn ttooonnnnttooonnnn

step: 1500, 
cost : 1.862301, 
prediction : f you want to  want tou want t   want ton want to  want to  tont to  uont to  u nt to  u l  to  u ld ao tu ld to lu ld a  lu ld a nlu ld a stutld a shutld a shi ld asshi fl anshi lo anssi l  anssi l dtnthi l donhhi l dontsi l don'si l don'tell don't fl don't al don't ar don't aruton't aru  n't aru  n t aru  u t aru  u t aru  u   aou  up ptou  up pe u  up pep   up pepp  up peppl up peopletp pepple   people tleeople totoople to lppee to  mree to  tnle to  thle to  theo to  thermto ether to ether t  ether tonnther to  ther to cm er to co er to colem to collm to colle to collecto lollect  lollectonlollectowtollectowotllectowoonlectowoodoectowood o towood amt wood anthaood and aood and tood and dond and dond and donn and don' and don'ttnt won't nt won't a  won't as aon't asston't ass  n't ass nn t ass nn t ass nn t ass nn t a s nn thtnsinn thenhinn themsinn them stttthem tfn them ta tthem tas t em tos toem tos s er tos s em tos s am tos s an uos s